# Day 16

## Part 1

In [69]:
with open("ex2.txt") as f:
    message = f.read().strip()
    
print(message)

38006F45291200


In [70]:
hex_to_bin = {"0": "0000", "1": "0001", "2": "0010", "3": "0011", "4": "0100", 
             "5": "0101", "6": "0110", "7": "0111", "8": "1000", "9": "1001", 
             "A": "1010", "B": "1011", "C": "1100", "D": "1101", "E": "1110", "F": "1111"}

new_message = "".join([hex_to_bin[char] for char in message])

print(new_message)

00111000000000000110111101000101001010010001001000000000


In [161]:
def parse_message(message):
    "Parses the whole of the message."
    print("starting!")
    
    # give this the original message
    rest_of_message, version_total, value = parse_packet(message, 0)
    # first gives in the whole message, then gets the same out but with the first packet taken off I think
    while "1" in rest_of_message:  # if there's only 0s left, then it's just padding
        rest_of_message, version_total, value2 = parse_packet(rest_of_message, version_total)
        value += value2
        
    return version_total, value

In [162]:
def parse_packet(message, version_total):
    """Parses the first packet from the message. Returns the rest of the message after the first packet.
    This includes parsing subpackets."""
    
    # All packets start with 6 bits (version number, type_ID).
    # Just handle these for now?
    version = message[:3]
    type_id = message[3:6]
    
    version_total += int(version, 2)    
    rest_of_message = ""
    
    if int(type_id, 2) == 4:
        # The first packet of message is a literal
        # parse this in the function which returns the number the literal represents
        # and then the remainder of the message that it didn't parse
        values = []
        value, rest_of_message = parse_literal(message[6:])
        values.append(value)
        # print("literal", number)
    else:
        # The first packet of message is an operator
        # This operator will contain some subpackets before it ends
        # After the operator, there may be more packets? Not sure if allowed
        # I need to get the subpackets out from each operator
        data = parse_operator(message[6:])
        # data could be a few things depending on length_type_id
        length_type_id = data[0]
        
        if length_type_id == "0":
            # Easiest case - data[1] is the bits for some number of subpackets of the operator
            subpackets = data[1]
            rest_of_message = data[2]  # this is the rest of the message after the subpackets
            
            # I need to parse the subpackets
            values = []
            sub_rest_of_message, version_total, value = parse_packet(subpackets, version_total)
            values.append(value)
            while sub_rest_of_message:
                sub_rest_of_message, version_total, value = parse_packet(sub_rest_of_message, version_total)  # there might be some padding left over I suppose?
                values.append(value)
                if "1" not in sub_rest_of_message:  # just 0s left over
                    break
        else:
            # Harder case - data[1] is how many subpackets there are
            # I need to parse the rest of the message until I've done n_subpackets worth
            # What do I do with the rest? Return it
            n_subpackets = data[1]
            rest_of_message = data[2]
            values = []

            for i in range(n_subpackets):
                # this should deal with returning the rest of the message each time
                rest_of_message, version_total, value = parse_packet(rest_of_message, version_total)
                values.append(value)
    
    # I now have a list of values
    # I need to return a true value, which depends on the type of packet this is
    packet_type = int(type_id, 2)
    
    if packet_type == 4:
        value = values[0]  # just a single number
    elif packet_type == 0:
        value = sum(values)  # sum packet
    elif packet_type == 1:
        value = 1  # product packet
        for i in values:
            value *= i
    elif packet_type == 2:
        value = min(values)
    elif packet_type == 3:
        value = max(values)
    elif packet_type == 5:
        value = int(values[0] > values[1])
    elif packet_type == 6:
        value = int(values[0] < values[1])
    elif packet_type == 7:
        value = int(values[0] == values[1])
        
    return rest_of_message, version_total, value

In [136]:
def parse_literal(message):
    # Starts from AFTER the version and type_id
    i = 0
    while True:
        if message[i*5] == "0":
            literal = message[:(i*5) + 5]  # the part of the message that's actually the number
            # suffix = message[(i*5) + 5:]  # the rest of the message
            break
        else:
            i += 1
            
    total_length = len(literal)
        
    this_packet = message[:total_length]
    rest_of_message = message[total_length:]
    
    # Go through literal, get binary sections
    number = []
    for j in range(i + 1):  # uses i from before for number of sections
        section = message[j*5:(j*5) + 5]
        number.append(section[1:])

    # print(int("".join(number), 2))
    
    return int("".join(number), 2), rest_of_message

In [138]:
def parse_operator(message):
    # first bit of this is length_type_id
    # do I need to deal with any padding with this stuff? IDK
    length_type_id = message[0]
    
    if length_type_id == "0":
        # 15 bits showing length of subpackets
        subpacket_length = int(message[1:16], 2)
        # get the subpackets
        subpackets = message[16: 16 + subpacket_length]
        rest_of_message = message[16 + subpacket_length:]
        return length_type_id, subpackets, rest_of_message  # return the subpacket strings to the original parse function?
    
    else:
        n_subpackets = int(message[1:12], 2)
        # this packet contains n_subpackets subpackets
        # But I don't know where they end, only that the next x packets count as being inside this packet
        # Return the rest of the message and the number of subpackets, and use them to deal with stuff I think
        rest_of_message = message[12:]  # I know that the rest of the message contains n_subpackets? hmm
        return length_type_id, n_subpackets, rest_of_message
        

In [163]:
for filename in ("ex1.txt", "ex2.txt", "ex3.txt", "ex4.txt", "ex5.txt", "ex6.txt", "ex7.txt"):

    with open(filename) as f:
        message = f.read().strip()

    print(message)

    hex_to_bin = {"0": "0000", "1": "0001", "2": "0010", "3": "0011", "4": "0100", 
                 "5": "0101", "6": "0110", "7": "0111", "8": "1000", "9": "1001", 
                 "A": "1010", "B": "1011", "C": "1100", "D": "1101", "E": "1110", "F": "1111"}

    new_message = "".join([hex_to_bin[char] for char in message])
    print(parse_message(new_message))
    print()

D2FE28
starting!
(6, 2021)

38006F45291200
starting!
(9, 1)

EE00D40C823060
starting!
(14, 3)

8A004A801A8002F478
starting!
(16, 15)

620080001611562C8802118E34
starting!
(12, 46)

C0015000016115A2E0802F182340
starting!
(23, 46)

A0016C880162017C3686B18A3D4780
starting!
(31, 54)



In [164]:
with open("message.txt") as f:
    message = f.read().strip()

print(message)

hex_to_bin = {"0": "0000", "1": "0001", "2": "0010", "3": "0011", "4": "0100", 
             "5": "0101", "6": "0110", "7": "0111", "8": "1000", "9": "1001", 
             "A": "1010", "B": "1011", "C": "1100", "D": "1101", "E": "1110", "F": "1111"}

new_message = "".join([hex_to_bin[char] for char in message])
print(parse_message(new_message))
print()

220D700071F39F9C6BC92D4A6713C737B3E98783004AC0169B4B99F93CFC31AC4D8A4BB89E9D654D216B80131DC0050B20043E27C1F83240086C468A311CC0188DB0BA12B00719221D3F7AF776DC5DE635094A7D2370082795A52911791ECB7EDA9CFD634BDED14030047C01498EE203931BF7256189A593005E116802D34673999A3A805126EB2B5BEEBB823CB561E9F2165492CE00E6918C011926CA005465B0BB2D85D700B675DA72DD7E9DBE377D62B27698F0D4BAD100735276B4B93C0FF002FF359F3BCFF0DC802ACC002CE3546B92FCB7590C380210523E180233FD21D0040001098ED076108002110960D45F988EB14D9D9802F232A32E802F2FDBEBA7D3B3B7FB06320132B0037700043224C5D8F2000844558C704A6FEAA800D2CFE27B921CA872003A90C6214D62DA8AA9009CF600B8803B10E144741006A1C47F85D29DCF7C9C40132680213037284B3D488640A1008A314BC3D86D9AB6492637D331003E79300012F9BDE8560F1009B32B09EC7FC0151006A0EC6082A0008744287511CC0269810987789132AC600BD802C00087C1D88D05C001088BF1BE284D298005FB1366B353798689D8A84D5194C017D005647181A931895D588E7736C6A5008200F0B802909F97B35897CFCBD9AC4A26DD880259A0037E49861F4E4349A6005CFAD180333E95281338A930EA400824981C

## Part 2

In [169]:
# examples

for i in range(8, 16):
    with open(f"ex{i}.txt") as f:
        message = f.read().strip()

    print(message)

    hex_to_bin = {"0": "0000", "1": "0001", "2": "0010", "3": "0011", "4": "0100", 
                 "5": "0101", "6": "0110", "7": "0111", "8": "1000", "9": "1001", 
                 "A": "1010", "B": "1011", "C": "1100", "D": "1101", "E": "1110", "F": "1111"}

    new_message = "".join([hex_to_bin[char] for char in message])
    print(parse_message(new_message))

C200B40A82
starting!
(14, 3)
04005AC33890
starting!
(8, 54)
880086C3E88112
starting!
(15, 7)
CE00C43D881120
starting!
(11, 9)
D8005AC2A8F0
starting!
(13, 1)
F600BC2D8F
starting!
(19, 0)
9C005AC2F8F0
starting!
(16, 0)
9C0141080250320F1802104A08
starting!
(20, 1)


In [170]:
with open("message.txt") as f:
    message = f.read().strip()

print(message)

hex_to_bin = {"0": "0000", "1": "0001", "2": "0010", "3": "0011", "4": "0100", 
             "5": "0101", "6": "0110", "7": "0111", "8": "1000", "9": "1001", 
             "A": "1010", "B": "1011", "C": "1100", "D": "1101", "E": "1110", "F": "1111"}

new_message = "".join([hex_to_bin[char] for char in message])
print(parse_message(new_message))

220D700071F39F9C6BC92D4A6713C737B3E98783004AC0169B4B99F93CFC31AC4D8A4BB89E9D654D216B80131DC0050B20043E27C1F83240086C468A311CC0188DB0BA12B00719221D3F7AF776DC5DE635094A7D2370082795A52911791ECB7EDA9CFD634BDED14030047C01498EE203931BF7256189A593005E116802D34673999A3A805126EB2B5BEEBB823CB561E9F2165492CE00E6918C011926CA005465B0BB2D85D700B675DA72DD7E9DBE377D62B27698F0D4BAD100735276B4B93C0FF002FF359F3BCFF0DC802ACC002CE3546B92FCB7590C380210523E180233FD21D0040001098ED076108002110960D45F988EB14D9D9802F232A32E802F2FDBEBA7D3B3B7FB06320132B0037700043224C5D8F2000844558C704A6FEAA800D2CFE27B921CA872003A90C6214D62DA8AA9009CF600B8803B10E144741006A1C47F85D29DCF7C9C40132680213037284B3D488640A1008A314BC3D86D9AB6492637D331003E79300012F9BDE8560F1009B32B09EC7FC0151006A0EC6082A0008744287511CC0269810987789132AC600BD802C00087C1D88D05C001088BF1BE284D298005FB1366B353798689D8A84D5194C017D005647181A931895D588E7736C6A5008200F0B802909F97B35897CFCBD9AC4A26DD880259A0037E49861F4E4349A6005CFAD180333E95281338A930EA400824981C